In [43]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [44]:
%%sql
drop table if exists Items,ShopItems;
CREATE TABLE Items
(item VARCHAR(16) PRIMARY KEY);
 
CREATE TABLE ShopItems
(shop VARCHAR(16),
 item VARCHAR(16),
    PRIMARY KEY(shop, item));

INSERT INTO Items VALUES('ビール');
INSERT INTO Items VALUES('紙オムツ');
INSERT INTO Items VALUES('自転車');

INSERT INTO ShopItems VALUES('仙台',  'ビール');
INSERT INTO ShopItems VALUES('仙台',  '紙オムツ');
INSERT INTO ShopItems VALUES('仙台',  '自転車');
INSERT INTO ShopItems VALUES('仙台',  'カーテン');
INSERT INTO ShopItems VALUES('東京',  'ビール');
INSERT INTO ShopItems VALUES('東京',  '紙オムツ');
INSERT INTO ShopItems VALUES('東京',  '自転車');
INSERT INTO ShopItems VALUES('大阪',  'テレビ');
INSERT INTO ShopItems VALUES('大阪',  '紙オムツ');
INSERT INTO ShopItems VALUES('大阪',  '自転車');

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [49]:
%%sql
select *
from ShopItems si
left join items it
on si.item = it.item

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


shop,item,item_1
仙台,ビール,ビール
仙台,紙オムツ,紙オムツ
仙台,自転車,自転車
仙台,カーテン,None
東京,ビール,ビール
東京,紙オムツ,紙オムツ
東京,自転車,自転車
大阪,テレビ,None
大阪,紙オムツ,紙オムツ
大阪,自転車,自転車


In [50]:
%%sql
-- havingを用いる
select si.shop
from ShopItems si
left join items it
on si.item = it.item
group by si.shop
-- (要求するitem) \supset (店舗にあるitem)
-- 仙台	カーテン	Noneなので'仙台'がNG
having count(si.item) = (
    select count(item) from items
) and
-- (要求するitem) \subset (店舗にあるitem)
-- 大阪	テレビ	Noneなので、'大阪'がNG
count(it.item) = (
    select count(item) from items
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


shop
東京
大阪
